### Webscrape the web to obtain sea port details

In [1]:
# load the library
import numpy as np
import pandas as pd
from tqdm import tqdm

# load selenium
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pyperclip
from IPython.display import clear_output
import sqlite3
from pandas.io import sql
import datetime
import time

In [2]:
# handling page refresh
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [3]:
# Set name of your SQLite database
db_name = "web_freight_data"

sqlite_db = db_name + '.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [4]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth',1000)
import time
from sklearn.externals import joblib

In [6]:
freight_lanes = pd.read_excel('freight_lanes.xlsx')

In [7]:
# download chromedriver.exe and save to a local drive locaiton:
location_of_chromedriver = "Selenium/chromedriver.exe"
site_url = "https://www.searates.com/maritime/"

In [10]:
# set browser and go to google maps
def init_browser(URL = 'https://www.google.com.hk/maps'):
    try:
        browser = webdriver.Chrome(location_of_chromedriver)
    except:
        browser = webdriver.Firefox()
    browser.get(URL)
    return browser

In [11]:
rate_site = init_browser(site_url)

In [102]:
countries = rate_site.find_elements_by_tag_name('a')

In [104]:
country_txt = rate_site.find_elements_by_class_name('col-xs-6')

In [107]:
link_list = []
country_list = []
for i in countries:
    tmp_txt = i.get_attribute('href')
    link_list.append(tmp_txt)
for i in country_txt:
    tmp_txt = i.text
    country_list.append(tmp_txt)

In [111]:
link_list2 = []
for i in link_list:
    try:
        if "/maritime/" in i and ".htm" in i:
            link_list2.append(i)
    except:
        pass

In [114]:
country_link = pd.DataFrame()
country_link['Country Links'] = link_list2
country_link['Country'] = country_list
country_link.to_csv('country_link.csv', index=False, encoding='utf_8')

In [115]:
country_link = pd.read_csv('country_link.csv')

In [124]:
def get_port_list(rate_site):
    port_list = rate_site.find_elements_by_tag_name('a')
    port_list2 = []
    for i in port_list:
        tmp = i.get_attribute('href')
        try:
            if "/port/" in tmp:
                port_list2.append(tmp)
        except:
            pass
    ports = []
    for i in range(len(port_list2)):
        tmp = rate_site.find_element_by_id('port_{}'.format(i)).text
        ports.append(tmp)
    return port_list2, ports

In [117]:
country_link.head()

,Country Links,Country
0,https://www.searates.com/maritime/albania.htm,Albania
1,https://www.searates.com/maritime/algeria.htm,Algeria
2,https://www.searates.com/maritime/american_samoa.htm,American Samoa
3,https://www.searates.com/maritime/andorra.htm,Andorra
4,https://www.searates.com/maritime/angola.htm,Angola


In [129]:
counter = 0
max_count = len(country_link)
summary_df = pd.DataFrame()

In [135]:
country_link['Country Links'].iloc[counter]

'https://www.searates.com/maritime/comoros.htm'

In [136]:
# find the individaul port detail links
while counter < max_count:
    rate_site.get(country_link['Country Links'].iloc[counter])
    time.sleep(1)
    tmp_link, tmp_port = get_port_list(rate_site)
    tmp_df = pd.DataFrame()
    tmp_df['Port Link'] = tmp_link
    tmp_df['Port'] = tmp_port
    tmp_df['Country'] = country_link['Country'].iloc[counter]
    summary_df = summary_df.append(tmp_df)
    counter = counter + 1

In [138]:
summary_df.shape

(3525, 3)

In [139]:
summary_df.to_csv('port_links.csv', index=False, encoding='utf_8')

### Review the summary details

In [5]:
summary_df = pd.read_csv('port_links.csv')
summary_df.head()

,Port Link,Port,Country
0,https://www.searates.com/port/durres_al.htm,Durres,Albania
1,https://www.searates.com/port/sarande_al.htm,Sarande,Albania
2,https://www.searates.com/port/shengjin_al.htm,Shengjin,Albania
3,https://www.searates.com/port/valona_al.htm,Valona,Albania
4,https://www.searates.com/port/vlore_al.htm,Vlore,Albania


In [43]:
def get_port_details(tmp_link):
    rate_site.get(tmp_link)
    td = rate_site.find_elements_by_tag_name('td')

    title = []
    element = []
    counter = 0
    while counter < len(td) - 1:
        tmp1 = td[counter].text
        tmp2 = td[counter + 1].text
        if ":" in tmp1:
            title.append(tmp1)
            element.append(tmp2)
            counter = counter + 2
        else:
            counter = counter + 1
    df = pd.DataFrame()
    for i in range(len(title)):
        df[title[i]] = [element[i]]
    return df

In [42]:
counter = 0
max_len = len(summary_df)

### Get the port details

In [47]:
while counter < max_len:
    clear_output()
    print("{} out of {}".format(counter, max_len))
    tmp_link = summary_df['Port Link'].iloc[counter]
    rate_site.get(tmp_link)
    time.sleep(1)
    tmp_df = get_port_details(tmp_link)
    tmp_df.to_sql('port_detail', con=conn, if_exists='append', index=False)
    counter = counter + 1

3524 out of 3525
